In [18]:
import os
import json
import nltk
import string
import numpy as np
import pandas as pd
from unidecode import unidecode
import transformers
import torch
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from tqdm import tqdm
from dotenv import load_dotenv

from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [19]:
load_dotenv()
os.environ['MISTRAL_API_KEY'] = os.getenv('MISTRAL_API_KEY')

## Setup device

In [20]:
device = (
  'cuda'
  if torch.cuda.is_available()
  else 'mps'
  if torch.backends.mps.is_available()
  else 'cpu'
)

## Setup metric

In [21]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(corpus):
  corpus = corpus.lower()
  stopset = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('russian') + list(string.punctuation)
  tokens = nltk.word_tokenize(corpus)
  tokens = [t for t in tokens if t not in stopset]
  tokens = [lemmatizer.lemmatize(t) for t in tokens]
  corpus = ' '.join(tokens)
  corpus = unidecode(corpus)
  return corpus

In [22]:
embeddings = OllamaEmbeddings(model='llama3')

In [23]:
def embeddings_cosine_sim_metric(expected_answers, predicted_answers):
  results = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    expected_embedding = np.array(embeddings.embed_query(expected_answer))
    predicted_embedding = np.array(embeddings.embed_query(predicted_answer))

    sim = cosine_similarity(
      expected_embedding.reshape(1, -1),
      predicted_embedding.reshape(1, -1),
    )[0][0]

    results.append(sim)

  return np.mean(results)

## Load QA dataset

In [24]:
qa_df = pd.read_csv('../research-neurobiology-qa-dataset/brainscape.csv')
qa_df

,question,answer
0,What are the afferent cranial nerve nuclei?,Trigeminal sensory nucleus- fibres carry gener...
1,What is the order of the cranial nerves ?,1-olfactory\n2-optic\n3-oculomotor\n4-trochlea...
2,What are the efferent cranial nerve nuclei?,Edinger-westphal nucleus\nOculomotor nucleus\n...
3,Which nuclei share the embryo logical origin -...,Oculomotor nucleus Trochlear nucleus Abducens ...
4,Which nuclei share the embryo logical origin- ...,Trigeminal motor nucleus Facial motor nucleus ...
...,...,...
1047,What is the purpose of gephyrin in the glycine...,Involved in anchoring the receptor to a specif...
1048,What is the glycine receptor involved in ?,Reflex response\nCauses reciprocal inhibition ...
1049,What happens in hyperperplexia ?,It’s an exaggerated reflex Often caused by a m...
1050,What is hyperperplexia treated with ?,Benzodiazepine


## Load documents

In [25]:
docs_dir = Path('./docs')
docs = []

for file in tqdm(docs_dir.iterdir()):
  if file.is_file() and file.suffix == '.pdf':
    loader = PDFMinerLoader(file)
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=700,
  chunk_overlap=0,
  length_function=len,
)

docs = text_splitter.split_documents(docs)

6it [08:46, 87.69s/it] 


## Setup LLMs

In [26]:
def get_llama2_llm():
  return Ollama(temperature=0, model='llama2')

In [27]:
def get_llama3_llm():
  return Ollama(temperature=0, model='llama3')

In [28]:
def openbiollm_parser(output):
  idx = output.find('Helpful Answer: ')
  if idx != -1:
    return output[idx + len('Helpful answer: '):]
  else:
    return output

def get_openbiollm_8b_llm():
  model = 'aaditya/OpenBioLLM-Llama3-8B'
  model_kwargs = {'torch_dtype': torch.bfloat16}
  pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    model_kwargs=model_kwargs,
    device=device,
  )
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids('<|eot_id|>')
  ]
  llm = HuggingFacePipeline.from_model_id(
    model_id=model,
    task='text-generation',
    model_kwargs=model_kwargs,
    pipeline_kwargs={
      'max_new_tokens': 256,
      'eos_token_id': terminators,
      'do_sample': True,
      'temperature': 0.0001,
      'top_p': 0.9,
    },
  )
  return llm | openbiollm_parser

In [29]:
def get_mistral_llm():
  return ChatMistralAI()

## Setup index stores

In [30]:
def get_doc_array_vector_store(docs=[]):
  index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
  ).from_documents(docs)
  return index.vectorstore

In [31]:
def get_chroma_vector_store(docs=[]):
  vector_store = Chroma.from_documents(docs, embeddings)
  return vector_store

## Setup experiments

In [32]:
llms = [
  ('LLaMA 2', get_llama2_llm()),
  ('LLaMA 3', get_llama3_llm()),
  ('OpenBioLLM Llama3 8B', get_openbiollm_8b_llm()),
  ('Mistral', get_mistral_llm()),
]

vector_stores = [
  ('DocArray', get_doc_array_vector_store),
  ('Chroma', get_chroma_vector_store),
]

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.15s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]
Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


In [33]:
cache_path = Path('cache.json')
with open(cache_path, 'r') as f:
  cache = json.load(f)

cache.keys()

dict_keys(['LLaMA 2_DocArray_False', 'LLaMA 2_DocArray_True', 'LLaMA 2_Chroma_True', 'LLaMA 3_DocArray_False', 'LLaMA 3_DocArray_True', 'LLaMA 3_Chroma_True', 'OpenBioLLM Llama3 8B_DocArray_False'])

In [34]:
df = pd.DataFrame(columns=['llm', 'vector_store', 'use_docs', 'accuracy'])

sample_df = qa_df.sample(frac=1)
questions = sample_df['question'].tolist()
expected_answers = sample_df['answer'].tolist()

for llm_name, llm in tqdm(llms, desc='LLMs'):
  for vector_store_name, get_vector_store in tqdm(vector_stores, desc='Vector Stores', leave=False):
    for use_docs in tqdm((False, True), desc='Use Docs', leave=False):
      if use_docs == False and vector_store_name != 'DocArray':
        continue
      vector_store = get_vector_store(docs)
      qa_llm = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=vector_store.as_retriever(search_kwargs={"k" : 10}),
        verbose=False,
        chain_type_kwargs = {'document_separator': '<<<<<>>>>>'},
      )

      predicted_answers = []

      for index, question in tqdm(enumerate(questions), desc='Questions', leave=False):
        key = f'{llm_name}_{vector_store_name}_{use_docs}'

        if not key in cache:
          cache[key] = {}
        if not question in cache[key]:
          cache[key][question] = qa_llm.invoke(question)['result']

        predicted_answers.append(cache[key][question])

        with open(cache_path, 'w') as f:
          json.dump(cache, f)

      accuracy = embeddings_cosine_sim_metric(expected_answers, predicted_answers)

      row = pd.DataFrame({
        'llm': llm_name,
        'vector_store': vector_store_name,
        'use_docs': use_docs,
        'accuracy': accuracy,
      }, index=[0])
      df = pd.concat([df, row], ignore_index=True)

LLMs:   0%|          | 0/4 [00:00<?, ?it/s]




































































































































































































































































































































































/tmp/ipykernel_4156820/3426733205.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row], ignore_index=True)










































































































































































































































































HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}

In [ ]:
df

,llm,vector_store,use_docs,accuracy
0,LLaMA 3,Doc Array In Memory Search,True,0.507332
1,LLaMA 3,Doc Array In Memory Search,False,0.444137
